# Tutorial: Mutual Timed Commitment Contract

In this tutorial, we will demonstrate the features of our tool using as example the Mutual Timed Commitment Contract.
Other examples (escrow contract, zero-coupon bond contract) can be found in the notebook [`docs/tutorial.ipnyb`](./tutorial.ipynb).


Table of contents:

- [Mutual Timed Commitment](#mutual-timed-commitment)
  - [Definition of the BitML contract](#definition-of-the-bitml-contract)
  - [Programmatic contract definition](#programmatic-contract-definition)
  - [Definition of the ATL specifications](#definition-of-the-atl-specifications)
    - [First specification](#first-specification)
    - [Second specification](#second-specification)
    - [Third specification](#third-specification)
  - [Translation to ISPL](#translation-to-ispl)
  - [ATL model checking with MCMAS](#atl-model-checking-with-mcmas) 
- [Flawed Mutual Timed Commitment](#flawed-mutual-timed-commitment)
- [Additional features](#additional-features)
  - [Generation of witnesses/counterexamples](#generation-of-witnessescounterexamples)  
  - [Simulation of the ISPL model](#simulation-of-the-ispl-model)
  - [Visualization of the model](#visualization-of-the-model)


## Mutual Timed Commitment

In the following, we will verify a BitML formalization of the _mutual timed commitment_ protocol.

### Definition of the BitML contract

First, we specify the contract in the BitML DSL. We reimplemented the parsing of the [original BitML toolchain](https://bitml-lang.github.io/nutshell.html) in the [Lark parsing toolkit](https://github.com/lark-parser/lark/). The Lark parser specification can be found at [this link](https://github.com/marcofavorito/bitml2mcmas/blob/main/bitml2mcmas/bitml/parser/grammars/main.lark).

In [1]:
bitml_mutual_timed_commitment = """
#lang bitml

(participant "A" "0a")
(participant "B" "0b")

(contract
  (pre
    (deposit "A" 1 "txA@0")
    (deposit "B" 1 "txB@0")
    (secret "A" a "0001a")
    (secret "B" b "0001b")
  )
  (choice
    (reveal (a) (choice
      (reveal (b) (split
        (1 -> (withdraw "A"))
        (1 -> (withdraw "B"))
      ))
      (after 2 (withdraw "A"))
    ))
    (after 1 (withdraw "B"))
  )
)
"""

Now, we will use the class `BitMLParser` to create an object representation of the BitML smart contract:

In [2]:
from bitml2mcmas.bitml.parser.parser import BitMLParser

# parse the contract
parser = BitMLParser()
contract = parser(bitml_mutual_timed_commitment)

We use the Lark parsing toolkit to generate the parser.

### Programmatic contract definition

Moreover, we can also programmatically define the same BitML smart contract, using the library APIs, as shown below.

The code for handling BitML is in the `bitml2mcmas.bitml` subpackage.

In [3]:
from decimal import Decimal

from bitml2mcmas.bitml.ast import BitMLParticipant, BitMLDepositPrecondition, BitMLTransactionOutput, \
    BitMLSecretPrecondition, BitMLChoiceExpression, BitMLRevealExpression, BitMLSplitBranch, BitMLAfterExpression, \
    BitMLWithdrawExpression, BitMLSplitExpression
from bitml2mcmas.bitml.core import BitMLContract

participant_a = BitMLParticipant(identifier='A', pubkey='0a')
participant_b = BitMLParticipant(identifier='B', pubkey='0b')
deposit_a = BitMLDepositPrecondition(
    participant_id='A',
    amount=Decimal('1'),
    tx=BitMLTransactionOutput(tx_identifier='txA', tx_output_index=0)
)
deposit_b = BitMLDepositPrecondition(
    participant_id='B',
    amount=Decimal('1'),
    tx=BitMLTransactionOutput(tx_identifier='txB', tx_output_index=0)
)
secret_a = BitMLSecretPrecondition(participant_id='A', secret_id='a', secret_hash='0001a')
secret_b = BitMLSecretPrecondition(participant_id='B', secret_id='b', secret_hash='0001b')


contract_code = BitMLChoiceExpression(choices=[
    BitMLRevealExpression(
            secret_ids=['a'],
            branch=BitMLChoiceExpression(choices=[
                BitMLRevealExpression(
                    secret_ids=['b'],
                    branch=BitMLSplitExpression(branches=[
                        BitMLSplitBranch(amount=Decimal('1'), branch=BitMLWithdrawExpression(participant_id='A')),
                        BitMLSplitBranch(amount=Decimal('1'), branch=BitMLWithdrawExpression(participant_id='B'))
                    ])),
                BitMLAfterExpression(
                    timeout=2,
                    branch=BitMLWithdrawExpression(participant_id='A'))
            ])),
    BitMLAfterExpression(timeout=1, branch=BitMLWithdrawExpression(participant_id='B'))
])


_contract = BitMLContract(
    participants=[
        participant_a,
        participant_b
    ],
    preconditions=[
        deposit_a,
        deposit_b,
        secret_a,
        secret_b
    ],
    contract=contract_code
)

### Definition of the ATL specifications

Next, we will declare the ${\rm ATL}$ specifications we aim to verify.


#### First specification

We start with the following:

- If participant $\texttt{A}$ publishes an invalid commitment, then he cannot recover his funds:

\\[ AG((\mathsf{private\_secret\_a\_is\_invalid} \to \lnot \langle A \rangle F (\mathsf{part\_A\_total\_deposits\_at\_least\_1}))
\\]

To build this ATL formula in our library, we require the definition of the two propositions:
- `private_secret_a_is_invalid`: the secret commitment for `a` is invalid
- `part_A_total_deposits_at_least_1`: the total of deposits held by participant $\texttt{A}$ is at least 1

In BitML2MCMAS, a proposition requires:
- an instance of `AtomicFormula`, used to build more complex formulae;
- an instance of `EvaluationRule`, which defines its semantics.

Below, the `Environment`'s variables `private_secret_a` and `part_A_total_deposits` are the names of the state variables generated by our translator:
- `private_secret_a`: the private state of the secret `a`, an enumeration of `{not_committed, valid, invalid}`;
- `part_A_total_deposits`: a variable that keeps track of the total deposits held by participant A (an integer).


In [4]:
from bitml2mcmas.mcmas.boolcond import EqualTo, EnvironmentIdAtom, IdAtom, GreaterThanOrEqual, IntAtom
from bitml2mcmas.mcmas.ast import EvaluationRule
from bitml2mcmas.mcmas.formula import AtomicFormula

########################################
# private_secret_a_is_invalid: the secret commitment for 'a' is invalid
# Definition: private_secret_a_is_invalid if (Agent_A.private_secret_a = invalid);
# The corresponding evaluation rule is added by default in the output ISPL program by our translator

PRIVATE_SECRET_A_IS_INVALID_PROP = "private_secret_a_is_invalid"
PRIVATE_SECRET_A_IS_INVALID = AtomicFormula(PRIVATE_SECRET_A_IS_INVALID_PROP)
########################################

########################################
# part_A_total_deposits_is_at_least_1: the total of deposits held by participant A is at least 1
# Definition: part_A_total_deposits_is_at_least_1 if (Environment.part_A_total_deposits >= 1);

# The name of the state variable for keeping track of the total deposits held by participant A, an integer
VAR_PART_A_TOTAL_DEPOSITS = "part_A_total_deposits"

PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_1_PROP = "part_A_total_deposits_is_at_least_1"
PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_1 = AtomicFormula(PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_1_PROP)
PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_1_ER = EvaluationRule(
    PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_1_PROP,
    GreaterThanOrEqual(EnvironmentIdAtom(VAR_PART_A_TOTAL_DEPOSITS), IntAtom(1))
)
########################################

Now, we can define the target ATL formula:

In [5]:
from bitml2mcmas.mcmas.formula import DiamondEventuallyFormula, AGFormula, ImpliesFormula, NotFormula

# AG((private_secret_a_is_invalid) -> (!<Agent_A>F(part_A_total_deposits_is_at_least_1)))
IF_AGENT_A_IS_DISHONEST_HE_DOES_NOT_RECOVER_FUNDS = AGFormula(
    ImpliesFormula(
        PRIVATE_SECRET_A_IS_INVALID,
        NotFormula(
            DiamondEventuallyFormula("Agent_A", PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_1)
        )
    )
)

#### Second specification

We consider other two ${\rm ATL}$ specifications:

- If the first timeout has expired and commitment for secret $\texttt{a}$ is invalid, then $\texttt{B}$ can recover his funds plus $\texttt{A}$'s penalty without revealing its secret $\texttt{b}$ (if it has not done it yet):

\\[
AG((
    \lnot\mathsf{public\_secret\_b\_is\_valid}
    \wedge \mathsf{timeout\_1\_expired}
    \wedge \mathsf{private\_secret\_a\_is\_invalid})
  \to \langle A \rangle F (
    \mathsf{part\_B\_total\_deposits\_is\_2}
    \wedge \lnot\mathsf{public\_secret\_b\_is\_valid}))
\\]


We require the definition of the following propositions:
- `public_secret_b_is_valid`: the secret `b` is published and it is the valid secret of the commitment;
- `timeout_1_expired`: the first timeout has expired;
- `private_secret_a_is_invalid`: see above;
- `part_B_total_deposits_is_at_least_2`:  the total of deposits held by participant $\texttt{B}$ is at least 2.


In [6]:
from bitml2mcmas.mcmas.formula import DiamondEventuallyFormula, AGFormula, ImpliesFormula, NotFormula

########################################
# public_secret_b_is_valid: the secret commitment for 'b' is valid
# Definition: public_secret_b_is_valid if (Environment.public_secret_b = valid);
# The corresponding evaluation rule is added by default in the output ISPL program by our translator

# The name of the state variable for the state of the public secret 'b', an enumeration of {not_committed, committed, valid}
VAR_PUBLIC_SECRET_B = "public_secret_b"

PUBLIC_SECRET_B_IS_VALID_PROP = "public_secret_b_is_valid"
PUBLIC_SECRET_B_IS_VALID = AtomicFormula(PUBLIC_SECRET_B_IS_VALID_PROP)
########################################

########################################
# timeout_1_has_expired: true iff the first timeout in the contract has expired
# Definition: timeout_1_has_expired if (Environment.time >= 1);
# The corresponding evaluation rule is added by default in the output ISPL program by our translator
TIMEOUT_1_HAS_EXPIRED_PROP = "timeout_1_has_expired"
TIMEOUT_1_HAS_EXPIRED = AtomicFormula(TIMEOUT_1_HAS_EXPIRED_PROP)
########################################

########################################
# part_B_total_deposits_is_2: the total of deposits held by participant B is equal to 2
# Definition: part_B_total_deposits_is_2 if (Environment.part_B_total_deposits = 2);

# The name of the state variable for keeping track of the total deposits held by participant B (an integer)
VAR_PART_B_TOTAL_DEPOSITS = "part_B_total_deposits"

PART_B_TOTAL_DEPOSITS_IS_2_PROP = "part_B_total_deposits_is_2"
PART_B_TOTAL_DEPOSITS_IS_2 = AtomicFormula(PART_B_TOTAL_DEPOSITS_IS_2_PROP)
PART_B_TOTAL_DEPOSITS_IS_2_ER = EvaluationRule(
    PART_B_TOTAL_DEPOSITS_IS_2_PROP,
    GreaterThanOrEqual(EnvironmentIdAtom(VAR_PART_B_TOTAL_DEPOSITS), IntAtom(2))
)
########################################

Finally, we can define the target ${\rm ATL}$ formula in code:

In [7]:
from bitml2mcmas.mcmas.formula import AndFormula

# AG((!public_secret_b_is_valid and private_secret_a_is_invalid and timeout_1_has_expired)
#     -> <Agent_B>F(!public_secret_b_is_valid and part_B_total_deposits_is_2))
IF_AGENT_A_IS_DISHONEST_AND_TIMEOUT_1_EXPIRED_THEN_B_CAN_RECOVER_FUNDS = AGFormula(
    ImpliesFormula(
        AndFormula(
            NotFormula(PUBLIC_SECRET_B_IS_VALID),
            AndFormula(
                PRIVATE_SECRET_A_IS_INVALID,
                TIMEOUT_1_HAS_EXPIRED
            )
        ),
        DiamondEventuallyFormula("Agent_B",
                                 AndFormula(
                                     NotFormula(PUBLIC_SECRET_B_IS_VALID),
                                     PART_B_TOTAL_DEPOSITS_IS_2
                                 )
        )
    )
)


#### Third specification

The third and last ${\rm ATL}$ specification that we will consider is:

- If participant $\texttt{B}$ commits to his secret and the first timeout has not expired yet, then it is guaranteed that $\texttt{B}$ always has a strategy to take at least 1 bitcoin:

\\[
AG((\mathsf{contract\_is\_initialized}
 \wedge \lnot\mathsf{timeout\_1\_expired}
 \wedge \mathsf{private\_secret\_b\_is\_valid})
\to \langle B\rangle F(\mathsf{part\_B\_total\_deposits\_is\_at\_least\_1}))
\\]


We require the definition of the following propositions:
- `contract_is_initialized`: this proposition is true if the contract has been initialized;
- `timeout_1_expired`: see above;
- `private_secret_b_is_valid`: the secret commitment for `b` is valid;
- `part_B_total_deposits_at_least_1`: the total of deposits held by participant $\texttt{B}$ is at least 1


In [8]:
########################################
# contract_is_initialized: true iff the contract is initialized or not in the current state
# Definition: contract_is_initialized if (Environment.contract_initialized = true);
# The corresponding evaluation rule is added by default in the output ISPL program by our translator
CONTRACT_IS_INITIALIZED_PROP = "contract_is_initialized"
CONTRACT_IS_INITIALIZED = AtomicFormula(CONTRACT_IS_INITIALIZED_PROP)
########################################


########################################
# private_secret_b_is_valid: the secret commitment for 'b' is valid
# Definition: private_secret_b_is_valid if (Agent_B.private_secret_b = valid);
# The corresponding evaluation rule is added by default in the output ISPL program by our translator

# The state of the private secret 'b', an enumeration of {not_committed, valid, invalid}
VAR_PRIVATE_SECRET_B = "private_secret_b"

PRIVATE_SECRET_B_IS_VALID_PROP = "private_secret_b_is_valid"
PRIVATE_SECRET_B_IS_VALID = AtomicFormula(PRIVATE_SECRET_B_IS_VALID_PROP)
########################################


########################################
# part_B_total_deposits_is_at_least_1: the total of deposits held by participant B is at least 1
# Definition: part_B_total_deposits_is_at_least_1 if (Environment.part_B_total_deposits >= 1);

# The name of the state variable for keeping track of the total deposits held by participant B, an integer
VAR_PART_B_TOTAL_DEPOSITS = "part_B_total_deposits"

PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_1_PROP = "part_B_total_deposits_is_at_least_1"
PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_1 = AtomicFormula(PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_1_PROP)
PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_1_ER = EvaluationRule(
    PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_1_PROP,
    GreaterThanOrEqual(EnvironmentIdAtom(VAR_PART_B_TOTAL_DEPOSITS), IntAtom(1))
)
########################################


The ${\rm ATL}$ formula in code is:

In [9]:
# AG((contract_is_initialized and private_secret_b_is_valid and !timeout_1_has_expired)
#    -> (<Agent_B>F(part_B_total_deposits_is_at_least_1)))
IF_BOTH_AGENT_A_AND_AGENT_B_HONEST_B_CAN_RECOVER_FUNDS = AGFormula(
    ImpliesFormula(
        AndFormula(
            CONTRACT_IS_INITIALIZED,
            AndFormula(
                PRIVATE_SECRET_B_IS_VALID,
                NotFormula(TIMEOUT_1_HAS_EXPIRED)
            )
        ),
        DiamondEventuallyFormula("Agent_B", PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_1),
    )
)

### Translation to ISPL

Finally, we are ready to produce the ISPL program file. To do so, we use instantiate the `Compiler` class with the contract, the formulae and the custom evaluation rules:


In [10]:
from bitml2mcmas.compiler.core import Compiler

formulae = [
    IF_AGENT_A_IS_DISHONEST_HE_DOES_NOT_RECOVER_FUNDS,
    IF_AGENT_A_IS_DISHONEST_AND_TIMEOUT_1_EXPIRED_THEN_B_CAN_RECOVER_FUNDS,
    IF_BOTH_AGENT_A_AND_AGENT_B_HONEST_B_CAN_RECOVER_FUNDS
]
evaluation_rules = [
    PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_1_ER,
    PART_B_TOTAL_DEPOSITS_IS_2_ER,
    PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_1_ER,
]
compiler = Compiler(
    contract, formulae, evaluation_rules=evaluation_rules
)


Next, we can call the `Compiler.compile()` function:

In [11]:
from pathlib import Path

interpreted_system = compiler.compile()

The result is an instance of the `InterpretedSystem` class, which represents an ISPL program as a Python object.

We can inspect the content of the model by accessing its attributes, e.g.:

In [12]:
interpreted_system.agents
interpreted_system.environment
interpreted_system.evaluation_rules
interpreted_system.initial_states_boolean_condition
interpreted_system.groups
interpreted_system.fair_formulae
interpreted_system.formulae;

The definitions of all other classes and functions for ISPL can be found in the `bitml2mcmas.mcmas` subpackage.

From the `InterpretedSystem` object, we can generate an ISPL file with the function `interpreted_system_to_string`

In [13]:
from bitml2mcmas.mcmas.to_string import interpreted_system_to_string

interpreted_system_str = interpreted_system_to_string(interpreted_system)
_ = Path("mutual-timed-commitment-tutorial-assets", "model.ispl").write_text(interpreted_system_str)

The output file `model.ispl` is pasted below:

```
Semantics=SingleAssignment;
Agent Environment
  Obsvars:
    time: 0..2;
    contract_funds: 0..2;
    part_A_total_deposits: 0..2;
    part_B_total_deposits: 0..2;
    public_secret_b: {committed, not_committed, valid};
    public_secret_a: {committed, not_committed, valid};
    contract_initialized: boolean;
    status_node_0_withdraw: {disabled, enabled, executed};
    status_node_1_withdraw: {disabled, enabled, executed};
    status_node_2_split: {disabled, enabled, executed};
    status_node_3_reveal: {disabled, enabled, executed};
    status_node_4_withdraw: {disabled, enabled, executed};
    status_node_6_reveal: {disabled, enabled, executed};
    status_node_7_withdraw: {disabled, enabled, executed};
  end Obsvars
  Vars:
    part_A_is_done: boolean;
    part_B_is_done: boolean;
    last_action: {action_delay, action_schedule_part_A, action_schedule_part_B, unset};
  end Vars

  Actions = {delay, schedule_part_A, schedule_part_B};
  Protocol:
    (part_A_is_done = false): {schedule_part_A};
    (part_B_is_done = false): {schedule_part_B};
    Other: {delay};
  end Protocol
  Evolution:
    part_A_is_done = false if ((Action = delay) and ((part_A_is_done = true) and (part_B_is_done = true)));
    part_B_is_done = false if ((Action = delay) and ((part_A_is_done = true) and (part_B_is_done = true)));
    part_A_is_done = true if (((Action = schedule_part_A) and (Agent_A.Action = nop)) and (part_A_is_done = false));
    part_B_is_done = true if (((Action = schedule_part_B) and (Agent_B.Action = nop)) and (part_B_is_done = false));
    time = (time + 1) if (((Action = delay) and ((part_A_is_done = true) and (part_B_is_done = true))) and (time < 2));
    public_secret_b = committed if (((public_secret_b = not_committed) and ((Agent_B.Action = commit_valid_secret_b) or (Agent_B.Action = commit_invalid_secret_b))) and (Action = schedule_part_B));
    public_secret_b = valid if (((Agent_B.Action = reveal_secret_b) and (Action = schedule_part_B)) and (public_secret_b = committed));
    public_secret_a = committed if (((public_secret_a = not_committed) and ((Agent_A.Action = commit_valid_secret_a) or (Agent_A.Action = commit_invalid_secret_a))) and (Action = schedule_part_A));
    public_secret_a = valid if (((Agent_A.Action = reveal_secret_a) and (Action = schedule_part_A)) and (public_secret_a = committed));
    contract_initialized = true if (((contract_initialized = false) and (((public_secret_b = committed) or (public_secret_b = valid)) and ((public_secret_a = committed) or (public_secret_a = valid)))) and (((Action = schedule_part_A) and (Agent_A.Action = initialize_contract)) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))));
    contract_funds = (contract_funds - 1) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_0_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_0_withdraw)));
    part_A_total_deposits = (part_A_total_deposits + 1) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_0_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_0_withdraw)));
    status_node_0_withdraw = executed if (((Agent_A.Action = exec_node_0_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_0_withdraw) and (Action = schedule_part_B)));
    status_node_0_withdraw = enabled if (((status_node_2_split = executed) or (((Agent_A.Action = exec_node_2_split) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_2_split) and (Action = schedule_part_B)))) and (status_node_0_withdraw = disabled));
    contract_funds = (contract_funds - 1) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_1_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_1_withdraw)));
    part_B_total_deposits = (part_B_total_deposits + 1) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_1_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_1_withdraw)));
    status_node_1_withdraw = executed if (((Agent_A.Action = exec_node_1_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_1_withdraw) and (Action = schedule_part_B)));
    status_node_1_withdraw = enabled if (((status_node_2_split = executed) or (((Agent_A.Action = exec_node_2_split) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_2_split) and (Action = schedule_part_B)))) and (status_node_1_withdraw = disabled));
    status_node_2_split = executed if (((Agent_A.Action = exec_node_2_split) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_2_split) and (Action = schedule_part_B)));
    status_node_2_split = enabled if (((status_node_3_reveal = executed) or (((Agent_A.Action = exec_node_3_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_3_reveal) and (Action = schedule_part_B)))) and (status_node_2_split = disabled));
    status_node_3_reveal = executed if (((Agent_A.Action = exec_node_3_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_3_reveal) and (Action = schedule_part_B)));
    status_node_3_reveal = enabled if (((((status_node_6_reveal = executed) or (((Agent_A.Action = exec_node_6_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_6_reveal) and (Action = schedule_part_B)))) and (status_node_3_reveal = disabled)) and ((public_secret_b = valid) or ((Agent_B.Action = reveal_secret_b) and (Action = schedule_part_B)))) and (!((status_node_4_withdraw = executed) or (((Agent_A.Action = exec_node_4_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_4_withdraw) and (Action = schedule_part_B))))));
    status_node_3_reveal = disabled if ((status_node_4_withdraw = executed) or (((Agent_A.Action = exec_node_4_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_4_withdraw) and (Action = schedule_part_B))));
    contract_funds = (contract_funds - 2) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_4_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_4_withdraw)));
    part_A_total_deposits = (part_A_total_deposits + 2) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_4_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_4_withdraw)));
    status_node_4_withdraw = executed if (((Agent_A.Action = exec_node_4_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_4_withdraw) and (Action = schedule_part_B)));
    status_node_4_withdraw = enabled if (((((status_node_6_reveal = executed) or (((Agent_A.Action = exec_node_6_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_6_reveal) and (Action = schedule_part_B)))) and (status_node_4_withdraw = disabled)) and ((time >= 2) or ((time = 1) and (Action = delay)))) and (!((status_node_3_reveal = executed) or (((Agent_A.Action = exec_node_3_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_3_reveal) and (Action = schedule_part_B))))));
    status_node_4_withdraw = disabled if ((status_node_3_reveal = executed) or (((Agent_A.Action = exec_node_3_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_3_reveal) and (Action = schedule_part_B))));
    status_node_6_reveal = executed if (((Agent_A.Action = exec_node_6_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_6_reveal) and (Action = schedule_part_B)));
    status_node_6_reveal = enabled if (((((((Action = schedule_part_A) and (Agent_A.Action = initialize_contract)) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))) or (contract_initialized = true)) and (status_node_6_reveal = disabled)) and ((public_secret_a = valid) or ((Agent_A.Action = reveal_secret_a) and (Action = schedule_part_A)))) and (!((status_node_7_withdraw = executed) or (((Agent_A.Action = exec_node_7_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_7_withdraw) and (Action = schedule_part_B))))));
    status_node_6_reveal = disabled if ((status_node_7_withdraw = executed) or (((Agent_A.Action = exec_node_7_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_7_withdraw) and (Action = schedule_part_B))));
    contract_funds = (contract_funds - 2) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_7_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_7_withdraw)));
    part_B_total_deposits = (part_B_total_deposits + 2) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_7_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_7_withdraw)));
    status_node_7_withdraw = executed if (((Agent_A.Action = exec_node_7_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_7_withdraw) and (Action = schedule_part_B)));
    status_node_7_withdraw = enabled if (((((((Action = schedule_part_A) and (Agent_A.Action = initialize_contract)) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))) or (contract_initialized = true)) and (status_node_7_withdraw = disabled)) and ((time >= 1) or ((time = 0) and (Action = delay)))) and (!((status_node_6_reveal = executed) or (((Agent_A.Action = exec_node_6_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_6_reveal) and (Action = schedule_part_B))))));
    status_node_7_withdraw = disabled if ((status_node_6_reveal = executed) or (((Agent_A.Action = exec_node_6_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_6_reveal) and (Action = schedule_part_B))));
    last_action = action_schedule_part_B if (Action = schedule_part_B);
    last_action = action_schedule_part_A if (Action = schedule_part_A);
    last_action = action_delay if (Action = delay);
  end Evolution
end Agent

Agent Agent_A
  Lobsvars = {part_A_is_done};
  Vars:
    private_secret_a: {invalid, not_committed, valid};
  end Vars

  Actions = {commit_invalid_secret_a, commit_valid_secret_a, exec_node_0_withdraw, exec_node_1_withdraw, exec_node_2_split, exec_node_3_reveal, exec_node_4_withdraw, exec_node_6_reveal, exec_node_7_withdraw, initialize_contract, nop, reveal_secret_a};
  Protocol:
    (((Environment.contract_initialized = false) and (private_secret_a = not_committed)) and (Environment.part_A_is_done = false)): {commit_invalid_secret_a, commit_valid_secret_a, nop};
    (((private_secret_a = valid) and (Environment.public_secret_a = committed)) and (Environment.part_A_is_done = false)): {nop, reveal_secret_a};
    (((Environment.contract_initialized = false) and (Environment.part_A_is_done = false)) and (((Environment.public_secret_b = committed) or (Environment.public_secret_b = valid)) and ((Environment.public_secret_a = committed) or (Environment.public_secret_a = valid)))): {initialize_contract, nop};
    ((Environment.status_node_0_withdraw = enabled) and (Environment.part_A_is_done = false)): {exec_node_0_withdraw, nop};
    ((Environment.status_node_1_withdraw = enabled) and (Environment.part_A_is_done = false)): {exec_node_1_withdraw, nop};
    ((Environment.status_node_2_split = enabled) and (Environment.part_A_is_done = false)): {exec_node_2_split, nop};
    ((Environment.status_node_3_reveal = enabled) and (Environment.part_A_is_done = false)): {exec_node_3_reveal, nop};
    ((Environment.status_node_4_withdraw = enabled) and (Environment.part_A_is_done = false)): {exec_node_4_withdraw, nop};
    ((Environment.status_node_6_reveal = enabled) and (Environment.part_A_is_done = false)): {exec_node_6_reveal, nop};
    ((Environment.status_node_7_withdraw = enabled) and (Environment.part_A_is_done = false)): {exec_node_7_withdraw, nop};
    Other: {nop};
  end Protocol
  Evolution:
    private_secret_a = valid if ((((Action = commit_valid_secret_a) and (private_secret_a = not_committed)) and (Environment.contract_initialized = false)) and (Environment.Action = schedule_part_A));
    private_secret_a = invalid if ((((Action = commit_invalid_secret_a) and (private_secret_a = not_committed)) and (Environment.contract_initialized = false)) and (Environment.Action = schedule_part_A));
  end Evolution
end Agent
Agent Agent_B
  Lobsvars = {part_B_is_done};
  Vars:
    private_secret_b: {invalid, not_committed, valid};
  end Vars

  Actions = {commit_invalid_secret_b, commit_valid_secret_b, exec_node_0_withdraw, exec_node_1_withdraw, exec_node_2_split, exec_node_3_reveal, exec_node_4_withdraw, exec_node_6_reveal, exec_node_7_withdraw, initialize_contract, nop, reveal_secret_b};
  Protocol:
    (((Environment.contract_initialized = false) and (private_secret_b = not_committed)) and (Environment.part_B_is_done = false)): {commit_invalid_secret_b, commit_valid_secret_b, nop};
    (((private_secret_b = valid) and (Environment.public_secret_b = committed)) and (Environment.part_B_is_done = false)): {nop, reveal_secret_b};
    (((Environment.contract_initialized = false) and (Environment.part_B_is_done = false)) and (((Environment.public_secret_b = committed) or (Environment.public_secret_b = valid)) and ((Environment.public_secret_a = committed) or (Environment.public_secret_a = valid)))): {initialize_contract, nop};
    ((Environment.status_node_0_withdraw = enabled) and (Environment.part_B_is_done = false)): {exec_node_0_withdraw, nop};
    ((Environment.status_node_1_withdraw = enabled) and (Environment.part_B_is_done = false)): {exec_node_1_withdraw, nop};
    ((Environment.status_node_2_split = enabled) and (Environment.part_B_is_done = false)): {exec_node_2_split, nop};
    ((Environment.status_node_3_reveal = enabled) and (Environment.part_B_is_done = false)): {exec_node_3_reveal, nop};
    ((Environment.status_node_4_withdraw = enabled) and (Environment.part_B_is_done = false)): {exec_node_4_withdraw, nop};
    ((Environment.status_node_6_reveal = enabled) and (Environment.part_B_is_done = false)): {exec_node_6_reveal, nop};
    ((Environment.status_node_7_withdraw = enabled) and (Environment.part_B_is_done = false)): {exec_node_7_withdraw, nop};
    Other: {nop};
  end Protocol
  Evolution:
    private_secret_b = valid if ((((Action = commit_valid_secret_b) and (private_secret_b = not_committed)) and (Environment.contract_initialized = false)) and (Environment.Action = schedule_part_B));
    private_secret_b = invalid if ((((Action = commit_invalid_secret_b) and (private_secret_b = not_committed)) and (Environment.contract_initialized = false)) and (Environment.Action = schedule_part_B));
  end Evolution
end Agent
Evaluation
  part_A_is_scheduled if (Environment.last_action = action_schedule_part_A);
  part_B_is_scheduled if (Environment.last_action = action_schedule_part_B);
  time_progresses_forever if ((Environment.time = 2) or (Environment.last_action = action_delay));
  time_reaches_maximum if (Environment.time = 2);
  timeout_1_has_expired if (Environment.time >= 1);
  timeout_2_has_expired if (Environment.time >= 2);
  part_A_is_done if (Environment.part_A_is_done = true);
  part_B_is_done if (Environment.part_B_is_done = true);
  private_secret_b_is_valid if (Agent_B.private_secret_b = valid);
  private_secret_b_is_invalid if (Agent_B.private_secret_b = invalid);
  private_secret_b_is_not_committed if (Agent_B.private_secret_b = not_committed);
  public_secret_b_is_committed if (Environment.public_secret_b = committed);
  public_secret_b_is_not_committed if (Environment.public_secret_b = not_committed);
  public_secret_b_is_valid if (Environment.public_secret_b = valid);
  private_secret_a_is_valid if (Agent_A.private_secret_a = valid);
  private_secret_a_is_invalid if (Agent_A.private_secret_a = invalid);
  private_secret_a_is_not_committed if (Agent_A.private_secret_a = not_committed);
  public_secret_a_is_committed if (Environment.public_secret_a = committed);
  public_secret_a_is_not_committed if (Environment.public_secret_a = not_committed);
  public_secret_a_is_valid if (Environment.public_secret_a = valid);
  contract_is_initialized if (Environment.contract_initialized = true);
  node_0_withdraw_is_disabled if (Environment.status_node_0_withdraw = disabled);
  node_0_withdraw_is_enabled if (Environment.status_node_0_withdraw = enabled);
  node_0_withdraw_is_executed if (Environment.status_node_0_withdraw = executed);
  node_1_withdraw_is_disabled if (Environment.status_node_1_withdraw = disabled);
  node_1_withdraw_is_enabled if (Environment.status_node_1_withdraw = enabled);
  node_1_withdraw_is_executed if (Environment.status_node_1_withdraw = executed);
  node_2_split_is_disabled if (Environment.status_node_2_split = disabled);
  node_2_split_is_enabled if (Environment.status_node_2_split = enabled);
  node_2_split_is_executed if (Environment.status_node_2_split = executed);
  node_3_reveal_is_disabled if (Environment.status_node_3_reveal = disabled);
  node_3_reveal_is_enabled if (Environment.status_node_3_reveal = enabled);
  node_3_reveal_is_executed if (Environment.status_node_3_reveal = executed);
  node_4_withdraw_is_disabled if (Environment.status_node_4_withdraw = disabled);
  node_4_withdraw_is_enabled if (Environment.status_node_4_withdraw = enabled);
  node_4_withdraw_is_executed if (Environment.status_node_4_withdraw = executed);
  node_6_reveal_is_disabled if (Environment.status_node_6_reveal = disabled);
  node_6_reveal_is_enabled if (Environment.status_node_6_reveal = enabled);
  node_6_reveal_is_executed if (Environment.status_node_6_reveal = executed);
  node_7_withdraw_is_disabled if (Environment.status_node_7_withdraw = disabled);
  node_7_withdraw_is_enabled if (Environment.status_node_7_withdraw = enabled);
  node_7_withdraw_is_executed if (Environment.status_node_7_withdraw = executed);
  part_A_total_deposits_is_at_least_1 if (Environment.part_A_total_deposits >= 1);
  part_B_total_deposits_is_2 if (Environment.part_B_total_deposits >= 2);
  part_B_total_deposits_is_at_least_1 if (Environment.part_B_total_deposits >= 1);
end Evaluation
InitStates
  (((((((((((((((((((Environment.time = 0) and (Environment.part_A_is_done = false)) and (Environment.part_B_is_done = false)) and (Environment.contract_funds = 2)) and (Environment.part_A_total_deposits = 0)) and (Environment.part_B_total_deposits = 0)) and (Environment.public_secret_b = not_committed)) and (Environment.public_secret_a = not_committed)) and (Agent_B.private_secret_b = not_committed)) and (Agent_A.private_secret_a = not_committed)) and (Environment.contract_initialized = false)) and (Environment.status_node_0_withdraw = disabled)) and (Environment.status_node_1_withdraw = disabled)) and (Environment.status_node_2_split = disabled)) and (Environment.status_node_3_reveal = disabled)) and (Environment.status_node_4_withdraw = disabled)) and (Environment.status_node_6_reveal = disabled)) and (Environment.status_node_7_withdraw = disabled)) and (Environment.last_action = unset));
end InitStates
Groups
  Participants = {Agent_A, Agent_B};
  Env = {Environment};
  ParticipantsAndEnv = {Agent_A, Agent_B, Environment};
  Agent_B = {Agent_B};
  Agent_A = {Agent_A};
end Groups
Fairness
  part_A_is_scheduled;
  part_B_is_scheduled;
  time_progresses_forever;
end Fairness
Formulae
  (AG ((private_secret_a_is_invalid) -> ((!(<Agent_A>F(part_A_total_deposits_is_at_least_1))))));
  (AG ((((!public_secret_b_is_valid) and (private_secret_a_is_invalid and timeout_1_has_expired))) -> ((<Agent_B>F(((!public_secret_b_is_valid) and part_B_total_deposits_is_2))))));
  (AG (((contract_is_initialized and (private_secret_b_is_valid and (!timeout_1_has_expired)))) -> ((<Agent_B>F(part_B_total_deposits_is_at_least_1)))));
end Formulae
```

### ATL model checking with MCMAS

We can run our verification task with the MCMAS model checker. By calling MCMAS with the command:

```
./mcmas -atlk 1  docs/mutual-timed-commitment-tutorial-assets/model.ispl
```

We get the following output:


```
************************************************************************
                       MCMAS v1.3.0 

 This software comes with ABSOLUTELY NO WARRANTY, to the extent
 permitted by applicable law. 

 Please check http://vas.doc.ic.ac.uk/tools/mcmas/ for the latest release.
 Please send any feedback to <mcmas@imperial.ac.uk>
************************************************************************

Command line: ./mcmas -atlk 1 docs/model.ispl

docs/model.ispl has been parsed successfully.
Global syntax checking...
1
1
1
Done
Encoding BDD parameters...
Building partial transition relation...
Building BDD for initial states...
Building reachable state space...
Checking formulae...
Building set of fair states...
Verifying properties...
  Formula number 1: (AG (private_secret_a_is_invalid -> (! (<Agent_A>F part_A_total_deposits_is_at_least_1)))), is TRUE in the model
  Formula number 2: (AG (((! public_secret_b_is_valid) && (private_secret_a_is_invalid && timeout_1_has_expired)) -> (<Agent_B>F ((! public_secret_b_is_valid) && part_B_total_deposits_is_2)))), is TRUE in the model
  Formula number 3: (AG ((contract_is_initialized && (private_secret_b_is_valid && (! timeout_1_has_expired))) -> (<Agent_B>F part_B_total_deposits_is_at_least_1))), is TRUE in the model
done, 3 formulae successfully read and checked
execution time = 1.808
number of reachable states = 1659
BDD memory in use = 14733968
```

## Flawed Mutual Timed Commitment

Here we consider the case where $t_1 \ge t_2$, for example, $t_1 = t_2 = 1$.
In particular, we replace `after 2` with `after 1`:

In [14]:
bitml_mutual_timed_commitment_flawed = """
#lang bitml

(participant "A" "0a")
(participant "B" "0b")

(contract
  (pre
    (deposit "A" 1 "txA@0")
    (deposit "B" 1 "txB@0")
    (secret "A" a "0001a")
    (secret "B" b "0001b")
  )
  (choice
    (reveal (a) (choice
      (reveal (b) (split
        (1 -> (withdraw "A"))
        (1 -> (withdraw "B"))
      ))
      (after 1 (withdraw "A"))
    ))
    (after 1 (withdraw "B"))
  )
)
"""
contract_flawed = parser(bitml_mutual_timed_commitment_flawed)
compiler_2 = Compiler(
    contract_flawed, formulae, evaluation_rules=evaluation_rules
)
interpreted_system_2 = compiler_2.compile()
interpreted_system_2_str = interpreted_system_to_string(interpreted_system_2)
_ = Path("mutual-timed-commitment-tutorial-assets", "model_flawed.ispl").write_text(interpreted_system_2_str)

By repeating the translation and the verification:

```
./mcmas -atlk 1 docs/mutual-timed-commitment-tutorial-assets/model_flawed.ispl
```

we get the following result:

```
************************************************************************
                       MCMAS v1.3.0

 This software comes with ABSOLUTELY NO WARRANTY, to the extent
 permitted by applicable law.

 Please check http://vas.doc.ic.ac.uk/tools/mcmas/ for the latest release.
 Please send any feedback to <mcmas@imperial.ac.uk>
************************************************************************

Command line: ./mcmas -atlk 1 -c 3 docs/mutual-timed-commitment-tutorial-assets/model_flawed.ispl

docs/mutual-timed-commitment-tutorial-assets/model_flawed.ispl has been parsed successfully.
Global syntax checking...
1
1
1
Done
Encoding BDD parameters...
Building partial transition relation...
Building BDD for initial states...
Building reachable state space...
Checking formulae...
Building set of fair states...
Verifying properties...
  Formula number 1: (AG (private_secret_a_is_invalid -> (! (<Agent_A>F part_A_total_deposits_is_at_least_1)))), is TRUE in the model
    -- Sorry it is not possible to compute witnesses for non-ECTLK formulae
  Formula number 2: (AG (((! public_secret_b_is_valid) && (private_secret_a_is_invalid && timeout_1_has_expired)) -> (<Agent_B>F ((! public_secret_b_is_valid) && part_B_total_deposits_is_2)))), is TRUE in the model
    -- Sorry it is not possible to compute witnesses for non-ECTLK formulae
  Formula number 3: (AG ((contract_is_initialized && (private_secret_b_is_valid && (! timeout_1_has_expired))) -> (<Agent_B>F part_B_total_deposits_is_at_least_1))), is FALSE in the model
  The following is a counterexample for the formula:
 OPERATOR CURRENTLY NOT SUPPORTED IN WITNESS/COUNTEREXAMPLEA counterexample exists but could not be generated.
done, 3 formulae successfully read and checked
execution time = 2.096
number of reachable states = 791
BDD memory in use = 15016176
```

In particular, note that now formula number 3 is not satisfied.

Here is the `diff` between `model.ispl` and `model_flawed.ispl`:

```
diff docs/mutual-timed-commitment-tutorial-assets/model.ispl docs/mutual-timed-commitment-tutorial-assets/model_flawed.ispl
```

```
4c4
<     time: 0..2;
---
>     time: 0..1;
36c36
<     time = (time + 1) if (((Action = delay) and ((part_A_is_done = true) and (part_B_is_done = true))) and (time < 2));
---
>     time = (time + 1) if (((Action = delay) and ((part_A_is_done = true) and (part_B_is_done = true))) and (time < 1));
58c58
<     status_node_4_withdraw = enabled if (((((status_node_6_reveal = executed) or (((Agent_A.Action = exec_node_6_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_6_reveal) and (Action = schedule_part_B)))) and (status_node_4_withdraw = disabled)) and ((time >= 2) or ((time = 1) and (Action = delay)))) and (!((status_node_3_reveal = executed) or (((Agent_A.Action = exec_node_3_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_3_reveal) and (Action = schedule_part_B))))));
---
>     status_node_4_withdraw = enabled if (((((status_node_6_reveal = executed) or (((Agent_A.Action = exec_node_6_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_6_reveal) and (Action = schedule_part_B)))) and (status_node_4_withdraw = disabled)) and ((time >= 1) or ((time = 0) and (Action = delay)))) and (!((status_node_3_reveal = executed) or (((Agent_A.Action = exec_node_3_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_3_reveal) and (Action = schedule_part_B))))));
127,128c127,128
<   time_progresses_forever if ((Environment.time = 2) or (Environment.last_action = action_delay));
<   time_reaches_maximum if (Environment.time = 2);
---
>   time_progresses_forever if ((Environment.time = 1) or (Environment.last_action = action_delay));
>   time_reaches_maximum if (Environment.time = 1);
130d129
<   timeout_2_has_expired if (Environment.time >= 2);
```



## Additional features 

### Generation of witnesses/counterexamples

For simple formulae (in particular, ECTLK formulae), MCMAS can generate _witnesses_ for true formulae and _counterexamples_ for false formulae.

Continuing with the mutual timed commitment contract, we consider two ${\rm CTL}$ specifications:

- There exist an execution path such that eventually the contract funds are zeroed:

\\[
EF(\mathsf{contract\_funds\_are\_zero})
\\]  

- For all execution paths, eventually on each path, the contract funds are zeroed:

\\[
AF(\mathsf{contract\_funds\_are\_zero})
\\]


In code:

In [15]:
from bitml2mcmas.mcmas.formula import EFFormula, AFFormula

########################################
# contract_funds_are_zero: true iff there are no more funds deposited in the contracts
# Definition: contract_funds_are_zero if (Environment.contract_funds = 0);

# The name of the state variable that keeps track of the funds currently held by the contract (an integer)
VAR_CONTRACT_FUNDS = "contract_funds"

CONTRACT_FUNDS_ARE_ZERO_PROP = "contract_funds_are_zero"
CONTRACT_FUNDS_ARE_ZERO = AtomicFormula(CONTRACT_FUNDS_ARE_ZERO_PROP)
CONTRACT_FUNDS_ARE_ZERO_ER = EvaluationRule(
    CONTRACT_FUNDS_ARE_ZERO_PROP, 
    EqualTo(EnvironmentIdAtom(VAR_CONTRACT_FUNDS), IntAtom(0))
)
########################################

# EF(contract_funds_are_zero)
EF_CONTRACT_FUNDS_ARE_ZERO = EFFormula(CONTRACT_FUNDS_ARE_ZERO)

# AF(contract_funds_are_zero)
AF_CONTRACT_FUNDS_ARE_ZERO = AFFormula(CONTRACT_FUNDS_ARE_ZERO)

compiler_3 = Compiler(
    contract, 
    [EF_CONTRACT_FUNDS_ARE_ZERO, AF_CONTRACT_FUNDS_ARE_ZERO], 
    evaluation_rules=[CONTRACT_FUNDS_ARE_ZERO_ER]
)
interpreted_system_3 = compiler_3.compile()
interpreted_system_3_str = interpreted_system_to_string(interpreted_system_3)
_ = Path("mutual-timed-commitment-tutorial-assets", "model_witness_counterexample.ispl").write_text(interpreted_system_3_str)

By repeating the translation and the verification with the options to generate witnesses and counterexamples:

```
./mcmas -p docs/mutual-timed-commitment-tutorial-assets/ -atlk 1 -c 3 docs/mutual-timed-commitment-tutorial-assets/model_witness_counterexample.ispl
```

we get the following output:


```
************************************************************************
                       MCMAS v1.3.0

 This software comes with ABSOLUTELY NO WARRANTY, to the extent
 permitted by applicable law.

 Please check http://vas.doc.ic.ac.uk/tools/mcmas/ for the latest release.
 Please send any feedback to <mcmas@imperial.ac.uk>
************************************************************************

Command line: ./mcmas -p docs/mutual-timed-commitment-tutorial-assets/ -atlk 1 -c 3 docs/mutual-timed-commitment-tutorial-assets/model_witness_counterexample.ispl

docs/mutual-timed-commitment-tutorial-assets/model_witness_counterexample.ispl has been parsed successfully.
Global syntax checking...
1
1
1
Done
Encoding BDD parameters...
Building partial transition relation...
Building BDD for initial states...
Building reachable state space...
Checking formulae...
Building set of fair states...
Verifying properties...
  Formula number 1: (EF contract_funds_are_zero), is TRUE in the model
  The following is a witness for the formula:
   < 0 1 2 3 4 5 6 7 >
  States description:
------------- State: 0 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = not_committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 0
  last_action = unset
  part_A_is_done = false
  part_B_is_done = false
Agent Agent_B
  private_secret_b = not_committed
Agent Agent_A
  private_secret_a = not_committed
----------------------------------------
------------- State: 1 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = not_committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 0
  last_action = action_schedule_part_B
  part_A_is_done = false
  part_B_is_done = true
Agent Agent_B
  private_secret_b = not_committed
Agent Agent_A
  private_secret_a = not_committed
----------------------------------------
------------- State: 2 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = not_committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 0
  last_action = action_schedule_part_A
  part_A_is_done = true
  part_B_is_done = true
Agent Agent_B
  private_secret_b = not_committed
Agent Agent_A
  private_secret_a = not_committed
----------------------------------------
------------- State: 3 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = not_committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 1
  last_action = action_delay
  part_A_is_done = false
  part_B_is_done = false
Agent Agent_B
  private_secret_b = not_committed
Agent Agent_A
  private_secret_a = not_committed
----------------------------------------
------------- State: 4 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 1
  last_action = action_schedule_part_A
  part_A_is_done = false
  part_B_is_done = false
Agent Agent_B
  private_secret_b = not_committed
Agent Agent_A
  private_secret_a = invalid
----------------------------------------
------------- State: 5 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = committed
  public_secret_b = committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 1
  last_action = action_schedule_part_B
  part_A_is_done = false
  part_B_is_done = false
Agent Agent_B
  private_secret_b = invalid
Agent Agent_A
  private_secret_a = invalid
----------------------------------------
------------- State: 6 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = true
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = committed
  public_secret_b = committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = enabled
  time = 1
  last_action = action_schedule_part_A
  part_A_is_done = false
  part_B_is_done = false
Agent Agent_B
  private_secret_b = invalid
Agent Agent_A
  private_secret_a = invalid
----------------------------------------
------------- State: 7 -----------------
Agent Environment
  contract_funds = 0
  contract_initialized = true
  part_A_total_deposits = 0
  part_B_total_deposits = 2
  public_secret_a = committed
  public_secret_b = committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = executed
  time = 1
  last_action = action_schedule_part_B
  part_A_is_done = false
  part_B_is_done = false
Agent Agent_B
  private_secret_b = invalid
Agent Agent_A
  private_secret_a = invalid
----------------------------------------
  Formula number 2: (AF contract_funds_are_zero), is FALSE in the model
  The following is a counterexample for the formula:
   < 0 1 2 3 4 5 6 7 8 9 7 >
  States description:
------------- State: 0 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = not_committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 0
  last_action = unset
  part_A_is_done = false
  part_B_is_done = false
Agent Agent_B
  private_secret_b = not_committed
Agent Agent_A
  private_secret_a = not_committed
----------------------------------------
------------- State: 1 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = not_committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 0
  last_action = action_schedule_part_B
  part_A_is_done = false
  part_B_is_done = true
Agent Agent_B
  private_secret_b = not_committed
Agent Agent_A
  private_secret_a = not_committed
----------------------------------------
------------- State: 2 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = not_committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 0
  last_action = action_schedule_part_A
  part_A_is_done = true
  part_B_is_done = true
Agent Agent_B
  private_secret_b = not_committed
Agent Agent_A
  private_secret_a = not_committed
----------------------------------------
------------- State: 3 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = not_committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 1
  last_action = action_delay
  part_A_is_done = false
  part_B_is_done = false
Agent Agent_B
  private_secret_b = not_committed
Agent Agent_A
  private_secret_a = not_committed
----------------------------------------
------------- State: 4 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = not_committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 1
  last_action = action_schedule_part_B
  part_A_is_done = false
  part_B_is_done = true
Agent Agent_B
  private_secret_b = not_committed
Agent Agent_A
  private_secret_a = not_committed
----------------------------------------
------------- State: 5 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 1
  last_action = action_schedule_part_A
  part_A_is_done = false
  part_B_is_done = true
Agent Agent_B
  private_secret_b = not_committed
Agent Agent_A
  private_secret_a = valid
----------------------------------------
------------- State: 6 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 1
  last_action = action_schedule_part_A
  part_A_is_done = true
  part_B_is_done = true
Agent Agent_B
  private_secret_b = not_committed
Agent Agent_A
  private_secret_a = valid
----------------------------------------
------------- State: 7 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 2
  last_action = action_delay
  part_A_is_done = false
  part_B_is_done = false
Agent Agent_B
  private_secret_b = not_committed
Agent Agent_A
  private_secret_a = valid
----------------------------------------
------------- State: 8 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 2
  last_action = action_schedule_part_B
  part_A_is_done = false
  part_B_is_done = true
Agent Agent_B
  private_secret_b = not_committed
Agent Agent_A
  private_secret_a = valid
----------------------------------------
------------- State: 9 -----------------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 2
  last_action = action_schedule_part_A
  part_A_is_done = true
  part_B_is_done = true
Agent Agent_B
  private_secret_b = not_committed
Agent Agent_A
  private_secret_a = valid
----------------------------------------
done, 2 formulae successfully read and checked
execution time = 0.588
number of reachable states = 1659
BDD memory in use = 12526128
```

As expected, we have that `EF(contract_funds_are_zero)` is true, while `AF(contract_funds_are_zero)` is false.
The MCMAS tool shows the sequence of states that make the formula to be satisfied (in the case of witnesses) or violated (in the case of counterexamples).


We can also visualize the witness by rendering the DOT file (the images are reported below):

```
dot -Tsvg docs/mutual-timed-commitment-tutorial-assets/formula1.dot -o docs/mutual-timed-commitment-tutorial-assets/formula1.svg
dot -Tsvg docs/mutual-timed-commitment-tutorial-assets/formula2.dot -o docs/mutual-timed-commitment-tutorial-assets/formula2.svg
```

The following is the visualization of the witness for the formula `EF(contract_funds_are_zero)`: in this witness, we have a sequence of actions of the three agents of the model, the `Environment`, `Agent_A` and `Agent_B`, which coordinate to make sure the funds in the contract are withdrawn. In particular, (1) first, the time is increased by one, so to make the first timeout to expire; (2) second, the contract is initialized; (3) third, the funds are withdrawn since timeout 1 has expired.

<img src="./mutual-timed-commitment-tutorial-assets/formula1.svg">


The following is the visualization of the counterexample for the formula `AF(contract_funds_are_zero)`: in this counterexample, the three agents coordinate so to never initialize the contract and withdraw funds, but only by executing "nop" actions, while satisfying our fairness constraints (all agents scheduled infinitely often, and time progressed infinitely often):

<img src="./mutual-timed-commitment-tutorial-assets/formula2.svg">


### Simulation of the ISPL model

We can use the MCMAS CLI tool to simulate the model, and so to debug the execution of the target BitML contract.

To do so, we can execute:

```
./mcmas -s docs/mutual-timed-commitment-tutorial-assets/model.ispl 
```

Then, it starts an interactive session where the user can provide input to select the action tuple (one action per agent).

In the proposed simulation of length 3, we take the following action tuples:

- `Environment : schedule_part_B; Agent_A : nop; Agent_B : commit_valid_secret_b`: the Environment schedules agent B, which commits to a valid secret;
- `Environment : schedule_part_A; Agent_A : commit_valid_secret_a; Agent_B : nop`: the Environment schedules agent A, which commits to a valid secret;
- `Environment : schedule_part_B; Agent_A : nop; Agent_B : initialize_contract`: the Environment schedules agent B, which initializes the contract. 

Follows the output of the tool:

```
************************************************************************
                       MCMAS v1.3.0

 This software comes with ABSOLUTELY NO WARRANTY, to the extent
 permitted by applicable law.

 Please check http://vas.doc.ic.ac.uk/tools/mcmas/ for the latest release.
 Please send any feedback to <mcmas@imperial.ac.uk>
************************************************************************

Command line: ./mcmas -s docs/mutual-timed-commitment-tutorial-assets/model.ispl

docs/mutual-timed-commitment-tutorial-assets/model.ispl has been parsed successfully.
Global syntax checking...
1
1
1
Done
Encoding BDD parameters...

--------- Initial state ---------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = not_committed
  public_secret_b = not_committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 0
  last_action = unset
  part_A_is_done = false
  part_B_is_done = false
Agent Agent_A
  private_secret_a = not_committed
Agent Agent_B
  private_secret_b = not_committed
----------------------------
Enabled actions:
1 : Environment : schedule_part_B; Agent_A : nop; Agent_B : commit_valid_secret_b
2 : Environment : schedule_part_B; Agent_A : nop; Agent_B : commit_invalid_secret_b
3 : Environment : schedule_part_B; Agent_A : commit_invalid_secret_a; Agent_B : commit_valid_secret_b
4 : Environment : schedule_part_B; Agent_A : nop; Agent_B : nop
5 : Environment : schedule_part_B; Agent_A : commit_valid_secret_a; Agent_B : commit_valid_secret_b
6 : Environment : schedule_part_B; Agent_A : commit_valid_secret_a; Agent_B : commit_invalid_secret_b
7 : Environment : schedule_part_B; Agent_A : commit_invalid_secret_a; Agent_B : commit_invalid_secret_b
8 : Environment : schedule_part_B; Agent_A : commit_invalid_secret_a; Agent_B : nop
9 : Environment : schedule_part_B; Agent_A : commit_valid_secret_a; Agent_B : nop
10 : Environment : schedule_part_A; Agent_A : nop; Agent_B : commit_invalid_secret_b
11 : Environment : schedule_part_A; Agent_A : commit_invalid_secret_a; Agent_B : commit_valid_secret_b
12 : Environment : schedule_part_A; Agent_A : commit_invalid_secret_a; Agent_B : nop
13 : Environment : schedule_part_A; Agent_A : nop; Agent_B : commit_valid_secret_b
14 : Environment : schedule_part_A; Agent_A : nop; Agent_B : nop
15 : Environment : schedule_part_A; Agent_A : commit_invalid_secret_a; Agent_B : commit_invalid_secret_b
16 : Environment : schedule_part_A; Agent_A : commit_valid_secret_a; Agent_B : nop
17 : Environment : schedule_part_A; Agent_A : commit_valid_secret_a; Agent_B : commit_valid_secret_b
18 : Environment : schedule_part_A; Agent_A : commit_valid_secret_a; Agent_B : commit_invalid_secret_b
Please choose one, or type 0 to backtrack or -1 to quit:
1

--------- Current state ---------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = not_committed
  public_secret_b = committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 0
  last_action = action_schedule_part_B
  part_A_is_done = false
  part_B_is_done = false
Agent Agent_A
  private_secret_a = not_committed
Agent Agent_B
  private_secret_b = valid
----------------------------
Enabled actions:
1 : Environment : schedule_part_B; Agent_A : nop; Agent_B : reveal_secret_b
2 : Environment : schedule_part_A; Agent_A : commit_valid_secret_a; Agent_B : nop
3 : Environment : schedule_part_B; Agent_A : nop; Agent_B : nop
4 : Environment : schedule_part_A; Agent_A : nop; Agent_B : nop
5 : Environment : schedule_part_A; Agent_A : commit_valid_secret_a; Agent_B : reveal_secret_b
6 : Environment : schedule_part_A; Agent_A : nop; Agent_B : reveal_secret_b
7 : Environment : schedule_part_B; Agent_A : commit_invalid_secret_a; Agent_B : nop
8 : Environment : schedule_part_A; Agent_A : commit_invalid_secret_a; Agent_B : nop
9 : Environment : schedule_part_B; Agent_A : commit_valid_secret_a; Agent_B : reveal_secret_b
10 : Environment : schedule_part_A; Agent_A : commit_invalid_secret_a; Agent_B : reveal_secret_b
11 : Environment : schedule_part_B; Agent_A : commit_invalid_secret_a; Agent_B : reveal_secret_b
12 : Environment : schedule_part_B; Agent_A : commit_valid_secret_a; Agent_B : nop
Please choose one, or type 0 to backtrack or -1 to quit:
2

--------- Current state ---------
Agent Environment
  contract_funds = 2
  contract_initialized = false
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = committed
  public_secret_b = committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 0
  last_action = action_schedule_part_A
  part_A_is_done = false
  part_B_is_done = false
Agent Agent_A
  private_secret_a = valid
Agent Agent_B
  private_secret_b = valid
----------------------------
Enabled actions:
1 : Environment : schedule_part_B; Agent_A : initialize_contract; Agent_B : nop
2 : Environment : schedule_part_B; Agent_A : reveal_secret_a; Agent_B : reveal_secret_b
3 : Environment : schedule_part_B; Agent_A : reveal_secret_a; Agent_B : nop
4 : Environment : schedule_part_B; Agent_A : initialize_contract; Agent_B : initialize_contract
5 : Environment : schedule_part_B; Agent_A : nop; Agent_B : reveal_secret_b
6 : Environment : schedule_part_B; Agent_A : initialize_contract; Agent_B : reveal_secret_b
7 : Environment : schedule_part_B; Agent_A : nop; Agent_B : initialize_contract
8 : Environment : schedule_part_A; Agent_A : initialize_contract; Agent_B : reveal_secret_b
9 : Environment : schedule_part_A; Agent_A : initialize_contract; Agent_B : nop
10 : Environment : schedule_part_A; Agent_A : initialize_contract; Agent_B : initialize_contract
11 : Environment : schedule_part_B; Agent_A : reveal_secret_a; Agent_B : initialize_contract
12 : Environment : schedule_part_B; Agent_A : nop; Agent_B : nop
13 : Environment : schedule_part_A; Agent_A : nop; Agent_B : initialize_contract
14 : Environment : schedule_part_A; Agent_A : nop; Agent_B : reveal_secret_b
15 : Environment : schedule_part_A; Agent_A : reveal_secret_a; Agent_B : reveal_secret_b
16 : Environment : schedule_part_A; Agent_A : nop; Agent_B : nop
17 : Environment : schedule_part_A; Agent_A : reveal_secret_a; Agent_B : nop
18 : Environment : schedule_part_A; Agent_A : reveal_secret_a; Agent_B : initialize_contract
Please choose one, or type 0 to backtrack or -1 to quit:
7

--------- Current state ---------
Agent Environment
  contract_funds = 2
  contract_initialized = true
  part_A_total_deposits = 0
  part_B_total_deposits = 0
  public_secret_a = committed
  public_secret_b = committed
  status_node_0_withdraw = disabled
  status_node_1_withdraw = disabled
  status_node_2_split = disabled
  status_node_3_reveal = disabled
  status_node_4_withdraw = disabled
  status_node_6_reveal = disabled
  status_node_7_withdraw = disabled
  time = 0
  last_action = action_schedule_part_B
  part_A_is_done = false
  part_B_is_done = false
Agent Agent_A
  private_secret_a = valid
Agent Agent_B
  private_secret_b = valid
----------------------------
Enabled actions:
1 : Environment : schedule_part_B; Agent_A : nop; Agent_B : reveal_secret_b
2 : Environment : schedule_part_A; Agent_A : reveal_secret_a; Agent_B : nop
3 : Environment : schedule_part_B; Agent_A : nop; Agent_B : nop
4 : Environment : schedule_part_A; Agent_A : reveal_secret_a; Agent_B : reveal_secret_b
5 : Environment : schedule_part_A; Agent_A : nop; Agent_B : reveal_secret_b
6 : Environment : schedule_part_A; Agent_A : nop; Agent_B : nop
7 : Environment : schedule_part_B; Agent_A : reveal_secret_a; Agent_B : reveal_secret_b
8 : Environment : schedule_part_B; Agent_A : reveal_secret_a; Agent_B : nop
Please choose one, or type 0 to backtrack or -1 to quit:
-1
```
 

### Visualization of the model

MCMAS can also render the ISPL model of our BitML contract in DOT format.


Since the graph of the mutual timed commitment contract would be too large, 
to exhibit this feature, we consider a simple contract where a participant $\texttt{A}$ wants to transfer 1 bitcoin to $\texttt{B}$.


In [16]:
bitml_simple_contract = """
#lang bitml

(participant "A" "0a")
(participant "B" "0b")

(contract
  (pre
    (deposit "A" 1 "txA@0")
  )
  (withdraw "B")
)
"""
contract_simple = parser(bitml_simple_contract)
compiler_simple = Compiler(
    contract_simple, [EF_CONTRACT_FUNDS_ARE_ZERO], evaluation_rules=[CONTRACT_FUNDS_ARE_ZERO_ER]
)
interpreted_system_simple = compiler_simple.compile()
interpreted_system_simple_str = interpreted_system_to_string(interpreted_system_simple)
_ = Path("mutual-timed-commitment-tutorial-assets", "model_simple.ispl").write_text(interpreted_system_simple_str)

To export the model, we can run:

```
./mcmas -exportmodel docs/mutual-timed-commitment-tutorial-assets/model_simple.ispl 
```

and then render the generated DOT file:

```
dot -Tsvg docs/mutual-timed-commitment-tutorial-assets/model.dot -o docs/mutual-timed-commitment-tutorial-assets/model_simpl
e.svg
```

Result:

<img src="./mutual-timed-commitment-tutorial-assets/model_simple.svg">